# 🔮 Lab 5: Semantic Kernel Agent
## Module 4 - Building Agents with Microsoft's Framework

**Duration:** 30 minutes

**Objectives:**
- Set up Semantic Kernel with Azure OpenAI
- Create plugins (tools) for banking operations
- Build a fraud investigation agent

In [ ]:
!pip install semantic-kernel openai -q
print("✅ Packages installed")

In [ ]:
import os, json

DEMO_MODE = False
MODEL_NAME = "gpt-4o"

try:
    from google.colab import userdata
    AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')
    AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')
    try: MODEL_NAME = userdata.get('AZURE_OPENAI_DEPLOYMENT')
    except: pass
    if AZURE_OPENAI_KEY and AZURE_OPENAI_ENDPOINT:
        if not AZURE_OPENAI_ENDPOINT.startswith('http'):
            AZURE_OPENAI_ENDPOINT = 'https://' + AZURE_OPENAI_ENDPOINT
        print(f"✅ Credentials loaded. Model: {MODEL_NAME}")
    else: raise ValueError()
except: print("⚠️ DEMO MODE"); DEMO_MODE = True

In [ ]:
if not DEMO_MODE:
    import semantic_kernel as sk
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
    
    kernel = sk.Kernel()
    kernel.add_service(
        AzureChatCompletion(
            service_id="azure-chat",
            deployment_name=MODEL_NAME,
            endpoint=AZURE_OPENAI_ENDPOINT,
            api_key=AZURE_OPENAI_KEY
        )
    )
    print("✅ Semantic Kernel initialized")
else:
    print("ℹ️ Semantic Kernel skipped (DEMO MODE)")

## Part 1: Create Banking Plugin

In [ ]:
if not DEMO_MODE:
    from semantic_kernel.functions import kernel_function
    
    class FraudInvestigationPlugin:
        @kernel_function(name="get_transaction_history", description="Get customer transactions")
        def get_transaction_history(self, customer_id: str) -> str:
            return json.dumps({"customer_id": customer_id, "avg_transaction": 2500, "max_transaction": 8000})
        
        @kernel_function(name="get_account_changes", description="Get recent account changes")
        def get_account_changes(self, customer_id: str) -> str:
            return json.dumps({"email_changed": "3 days ago", "phone_changed": "3 days ago"})
        
        @kernel_function(name="get_login_history", description="Get login history")
        def get_login_history(self, customer_id: str) -> str:
            return json.dumps({"usual_location": "San Francisco", "recent_login": "Lagos, Nigeria"})
    
    kernel.add_plugin(FraudInvestigationPlugin(), plugin_name="fraud")
    print("✅ Plugin registered")
else:
    print("ℹ️ Plugin demo - would register FraudInvestigationPlugin with tools for transaction history, account changes, login history")

## Part 2: Run Investigation

In [ ]:
alert = """FRAUD ALERT - Customer C-789
Transaction: $45,000 wire to Nigeria
Flagged: Amount threshold + International"""

if not DEMO_MODE:
    from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
    from semantic_kernel.contents.chat_history import ChatHistory
    
    async def investigate(alert):
        chat = ChatHistory()
        chat.add_system_message("You are a fraud investigator. Use tools to gather info.")
        chat.add_user_message(f"Investigate: {alert}")
        
        settings = kernel.get_prompt_execution_settings_from_service_id("azure-chat")
        settings.function_choice_behavior = FunctionChoiceBehavior.Auto()
        
        service = kernel.get_service("azure-chat")
        result = await service.get_chat_message_content(chat_history=chat, settings=settings, kernel=kernel)
        return result
    
    result = await investigate(alert)
    print(f"Result: {result}")
else:
    print(f"\n🔮 DEMO: Semantic Kernel Investigation")
    print(f"Alert: {alert}")
    print("\nSimulated flow:")
    print("  1. Agent calls get_transaction_history → avg $2,500, max $8,000")
    print("  2. Agent calls get_account_changes → email/phone changed 3 days ago")
    print("  3. Agent calls get_login_history → login from Lagos, Nigeria")
    print("\n📋 RECOMMENDATION: BLOCK - High risk indicators")

## Semantic Kernel vs Raw SDK

| Aspect | Semantic Kernel | Raw SDK |
|--------|-----------------|--------|
| Tool Definition | @kernel_function | Manual JSON |
| Tool Execution | Automatic | Manual dispatch |
| Type Safety | Python hints | None |
| Plugin Organization | Class-based | Flat list |

## ✅ Lab 5 Complete!

**Key Takeaways:**
- Semantic Kernel provides structured agent building
- Plugins organize tools into logical groups
- @kernel_function simplifies tool definition

**Next:** `06_azure_ai_foundry.ipynb`